In [44]:
from bs4 import BeautifulSoup
import requests

url = "http://web.archive.org/web/20110514112442/http://unstats.un.org/unsd/demographic/products/socind/education.htm"

r = requests.get(url)

In [45]:
import pandas as pd
soup = BeautifulSoup(r.content)
soup = soup('table')[6]
df = pd.DataFrame(columns=['Country','Year','Total','Men','Women'])
j = 0
for i in soup.findAll('tr')[8:-11]:
    col = i.findAll('td')
    #print col[0].text
    #print col[1].text
    #print col[4].text
    #print col[7].text
    #print col[10].text
    df.loc[j] = [col[0].text,col[1].text,col[4].text,col[7].text,col[10].text]
    j += 1    

In [46]:
df.set_index('Country',inplace=True)

In [4]:
import numpy as np
df['Men'] = df['Men'].astype(np.int64)
df['Women'] = df['Women'].astype(np.int64)
df['Total'] = df['Total'].astype(np.int64)

In [47]:
df

,Year,Total,Men,Women
Country,,,,
Afghanistan,2004,8,11,5
Albania,2004,11,11,11
Algeria,2005,13,13,13
Andorra,2008,11,11,12
Anguilla,2008,11,11,11
Antigua and Barbuda,2000,14,12,16
Argentina,2007,16,15,17
Armenia,2009,12,12,13
Aruba,2009,13,13,13


In [36]:
import sqlite3 as lite

con = lite.connect('education.db')
cur = con.cursor()

df.reset_index()

df.to_sql(con=con, name='school_life', if_exists='replace', flavor='sqlite')

In [41]:
import csv

#con = lite.connect('gdp.db')
#cur = con.cursor()

with open('37471cbd-1b7d-43a0-ba94-e55c3c7fc0a5_v2.csv','rU') as inputFile:
    next(inputFile) # skip the first 3 lines
    next(inputFile)
    next(inputFile)
    header = next(inputFile)
    inputReader = csv.reader(inputFile)
    columns = next(inputReader)[43:-5]  
    columns = ["country_name"] + ["_" + year for year in columns]
#    print columns
    inputs = ["country_name TEXT"] + [column + " NUMERIC" for column in columns[1:]]
#    print inputs
    cur.execute("DROP TABLE IF EXISTS gdp")
    cur.execute("CREATE TABLE gdp " + "(" + ", ".join(inputs) + ")")
    for line in inputReader:
        with con:
            country = line[0]
            gdp_vals = line[43:-5]
            
            cur.execute("INSERT INTO gdp " + "(" + ", ".join(columns) + ") " +
                        'VALUES ("' + country + '","' + '","'.join(gdp_vals) + '");')

In [42]:
gdp_df = pd.read_sql_query("SELECT * FROM gdp ORDER BY country_name", con)
#, index_col = "country_name")
#gdp_df.head()

In [43]:
gdp_df.dtypes
gdp_df2 = pd.melt(gdp_df, id_vars=["country_name"], var_name="Year", value_name="gdp")

In [24]:
gdp_df2

,country_name,Year,gdp
0,Afghanistan,_1999,
1,Albania,_1999,3.41476e+09
2,Algeria,_1999,4.86406e+10
3,American Samoa,_1999,
4,Andorra,_1999,1.23984e+09
5,Angola,_1999,6.15294e+09
6,Antigua and Barbuda,_1999,6.51852e+08
7,Arab World,_1999,6.40059e+11
8,Argentina,_1999,283523000000
9,Armenia,_1999,1.84548e+09


In [19]:
#get list of countries for which we have both data on GDP and life expectancy
ed_country_list = df.index.values.tolist()
gdp_country_list = gdp_df[gdp_df != ""].dropna().index.values.tolist()
country_list = list(set(ed_country_list).intersection(gdp_country_list))

In [20]:
gdp_country_list

[u'Albania',
 u'Algeria',
 u'Andorra',
 u'Angola',
 u'Antigua and Barbuda',
 u'Arab World',
 u'Argentina',
 u'Armenia',
 u'Aruba',
 u'Australia',
 u'Austria',
 u'Azerbaijan',
 u'Bahamas, The',
 u'Bahrain',
 u'Bangladesh',
 u'Barbados',
 u'Belarus',
 u'Belgium',
 u'Belize',
 u'Benin',
 u'Bermuda',
 u'Bhutan',
 u'Bolivia',
 u'Bosnia and Herzegovina',
 u'Botswana',
 u'Brazil',
 u'Brunei Darussalam',
 u'Bulgaria',
 u'Burkina Faso',
 u'Burundi',
 u'Cabo Verde',
 u'Cambodia',
 u'Cameroon',
 u'Canada',
 u'Caribbean small states',
 u'Central African Republic',
 u'Central Europe and the Baltics',
 u'Chad',
 u'Chile',
 u'China',
 u'Colombia',
 u'Comoros',
 u'Congo, Dem. Rep.',
 u'Congo, Rep.',
 u'Costa Rica',
 u"Cote d'Ivoire",
 u'Croatia',
 u'Cuba',
 u'Cyprus',
 u'Czech Republic',
 u'Denmark',
 u'Djibouti',
 u'Dominica',
 u'Dominican Republic',
 u'East Asia & Pacific (all income levels)',
 u'East Asia & Pacific (developing only)',
 u'Ecuador',
 u'Egypt, Arab Rep.',
 u'El Salvador',
 u'Equatoria